In [1]:
from pathlib import Path
from copy import deepcopy

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from mersearch.helpers import smiles_to_mol_file_format

In [2]:
profile_path = str(Path('~') / 'Library' / 'Application Support' / 'Firefox' / 'Profiles' / 'pwofczxw.default')
PROXY_HOST = "12.12.12.123"
PROXY_PORT = "1234"
options=Options()
options.set_preference('profile', profile_path)
options.set_preference("network.proxy.type", 1)
options.set_preference("network.proxy.http", PROXY_HOST)
options.set_preference("network.proxy.http_port", int(PROXY_PORT))
options.set_preference("dom.webdriver.enabled", False)
options.set_preference('useAutomationExtension', False)
options.set_preference("excludeSwitches", "enable-automation")
service = Service(r'/usr/local/bin/geckodriver')

driver = webdriver.Firefox(service=service, options=options)

In [48]:
url = "https://www.thermofisher.com/search/chemical/structure"
driver.get(url)

In [49]:
# find iframe for marvinjs
WebDriverWait(driver=driver, timeout=1).until(EC.presence_of_element_located((By.ID, 'sketch')))
driver.switch_to.frame('sketch')

In [50]:
# find toolbar of buttons
mjs_toolbar = driver.find_element(By.CLASS_NAME, 'mjs-toolbar').find_element(By.CLASS_NAME, 'mjs-toolbar')
# clear the page
mjs_toolbar.find_element(By.XPATH, './/td[1]').find_element(By.TAG_NAME, 'img').click()
# open mol file load
mjs_toolbar.find_element(By.XPATH, './/td[2]').find_element(By.TAG_NAME, 'img').click()

In [51]:
# add mol file to the pop up
popup = (
    driver.find_element(By.CLASS_NAME, 'gwt-DialogBox.mjs-top')
    .find_element(By.CLASS_NAME, 'dialogMiddleCenter')
)
molfile_textarea = (
    popup.find_element(By.XPATH, './/tr[1]')
    .find_element(By.XPATH, './/tr[6]')
    .find_element(By.TAG_NAME, 'textarea')
)
# add molfile data
# smiles = 'O=C1CCCCCCCCCCCCCCO1'
# smiles = 'O=COC(=O)C1(C)CCCC2(C)C3CC(=O)OCC3CCC12'
smiles = "OCC"
molfile_textarea.clear()
molfile_textarea.send_keys(smiles_to_mol_file_format(smiles))

In [52]:
# push button to add
popup.find_element(By.XPATH, './/tr[2]/td/div/div[1]/button').click()

In [53]:
# switch out of iframe
driver.switch_to.default_content()

In [54]:
driver.find_element(By.ID, 'exact-structure').click()

In [55]:
# click outside to exit if an error occurred
action = ActionChains(driver)
action.move_by_offset(0, 0).click().perform()